# Instruccions

L'exàmen consta de dos exercicis (4 punts cada un) i una pregunta (2 punts). 

Per a poder fer els exercicis, heu de descarregar el fitxer `data.csv` i el fitxer `practica2.py` del campus virtual.

`IMPORTANT`: Per calcular la semblança entre dos usuaris heu de fer servir le funció `similarity_matrix_2`.

In [1]:
import numpy as np
import pandas as pd
import pickle as pk

import practica2 as p2

data = pd.read_csv("data.csv")
num_movies = data.nunique()['movie_id']

### Exercici 1

Executa el recomanador (fent servir el mètode `similarity_matrix_2`) per trobar les recomanacions per a un usuari amb `user_id = 123`, que tingui en compte els 150 usuaris més semblants a ell i retorni 5 pel·lícules. Processa la sortida del recomanador (si és necessari) perquè només mostri les pel·lícules que creu que tenen una puntuació superior a 3.5.

In [2]:
# Operacions pesades les faig apart si puc per estalviar temps
df_counts = p2.build_counts_table(data)
sim_matrix = p2.similarity_matrix_2(df_counts)

In [10]:
user_id = 123  # usuari per al qual fem les recomanacions

# preparem les dades necessaries per a obtenir les recomanacions
df_counts = p2.build_counts_table(data)
sim_matrix = p2.similarity_matrix_2(df_counts)

# cridem al recomandador
user_recomendations = p2.getRecommendationsUser(data, user_id, sim_matrix, 5, 150)

recomendation_mask = user_recomendations['predicted_score'] > 3.5
# filtrem per al rating que se'ns demana
recomendations = user_recomendations[recomendation_mask]

# calculem la resposta i la mostrem
ids = recomendations['movie_id']

# Es podria fer més modular amb un for on hi hagues un parametre, s on s= #pelicules i es fes append a names
name1 = data[data['movie_id'] == ids[0]]['title'].iloc[0]
name2 = data[data['movie_id'] == ids[1]]['title'].iloc[0]
name3 = data[data['movie_id'] == ids[2]]['title'].iloc[0]
name4 = data[data['movie_id'] == ids[3]]['title'].iloc[0]
name5 = data[data['movie_id'] == ids[4]]['title'].iloc[0]
names = [name1, name2, name3, name4, name5]
print(names)

['Babe (1995)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 'Blade Runner (1982)', 'Citizen Kane (1941)', 'Being John Malkovich (1999)']


In [11]:
# --- 1. Configuració de paràmetres ---
target_user_id = 123
k_neighbors = 150      # Nombre de veïns a considerar
n_recs_requested = 5   # Nombre de recomanacions inicials a demanar
min_score = 3.5        # Llindar de tall

# --- 2. Càlcul de Matrius ---
# Construïm la taula de freqüències/vots
df_counts = p2.build_counts_table(data)

# Calculem la matriu de similitud fent servir la versió 2 (segons enunciat)
sim_matrix = p2.similarity_matrix_2(df_counts)

# --- 3. Generació de Recomanacions ---
# Cridem la funció per obtenir les prediccions brutes
# Assumim que la funció retorna un DataFrame amb columnes ['movie_id', 'predicted_score']
raw_recs = p2.getRecommendationsUser(
    data, 
    target_user_id, 
    sim_matrix, 
    n_recs_requested, 
    k_neighbors
)

# --- 4. Processament: Filtratge i Neteja ---
# Pas crític: Filtrem per score > 3.5
# Nota: Si el recomanador retorna 5 pel·lícules i cap supera el 3.5, el resultat serà buit.
filtered_recs = raw_recs[raw_recs['predicted_score'] > min_score].copy()

# Ens assegurem que estiguin ordenades de major a menor puntuació
filtered_recs = filtered_recs.sort_values(by='predicted_score', ascending=False)

# --- 5. Visualització (Obtenció de Títols) ---
# Fem un merge amb el dataset original per recuperar els títols
# drop_duplicates() és vital per no duplicar files si 'data' té múltiples entrades per movie_id
movies_metadata = data[['movie_id', 'title']].drop_duplicates()

final_result = filtered_recs.merge(movies_metadata, on='movie_id', how='left')

# Mostrem resultat
print(f"Recomanacions per l'usuari {target_user_id} (Score > {min_score}):")
print(final_result[['movie_id', 'title', 'predicted_score']])

Recomanacions per l'usuari 123 (Score > 3.5):
   movie_id                                              title  \
0        34                                        Babe (1995)   
1       750  Dr. Strangelove or: How I Learned to Stop Worr...   
2       541                                Blade Runner (1982)   
3       923                                Citizen Kane (1941)   
4      2997                        Being John Malkovich (1999)   

   predicted_score  
0         3.974887  
1         3.935489  
2         3.897483  
3         3.876334  
4         3.834767  


In [ ]:
# Sempre que fas una condició booleana dins un df amb df[condicio] lo de dins és una amscara booleana 
# (compares una part del df amb una codnició) i agafa totes les dades dels que son true
# Les mascares es poden "sumar"

mask_score = df['score'] > 3.5
mask_genre = df['genre'] == 'Terror'
final_mask = mask_score & mask_genre  # Combinació lògica

filtered = df[final_mask].copy()

Sense .copy(): Pandas et retorna una vista (view) de les dades originals. Si intentes modificar filtered_recs després, Pandas et llançarà el temut SettingWithCopyWarning, perquè no sap si vols modificar el trosretallat o l'original.

    Amb .copy(): Forces la creació d'un nou objecte a memòria. A partir d'aquí, filtered_recs és independent i pots modificar-lo sense problemes.

    [!tip] Professor's Tip Sempre que creïs un sub-dataframe que pensis modificar més endavant (ordenar, afegir columnes, etc.), posa-hi un .copy() al final. T'estalviaràs molts mals de cap.

### Exercici 2

Una mètrica alternativa d'evaluació per recomanadors és la **Mean Absolute Percentage Error** (MAPE). Aquesta mètrica es defineix com: 

$$ MAPE = \frac{1}{n} \sum_{i=1}^{n} \left| \frac{A_i - F_i}{A_i} \right| $$

On $A_i$ és el valor real, $F_i$ és el valor pronosticat i n és el nombre total de prediccions.

Per aplicar-la, crearem un dataset nou més petit (veure la cel·la de sota). 
A partir d'aquí:

1. Recalcula la matriu de similitud amb aquest nou dataset.
2. Divideix el dataset en un conjunt d'entrenament (90%) i un de test (10%).
3. Implementa la funció `evaluateRecommendations_mape(train, test, m,n, sim)`
4. Avalua el recomanador amb aquesta nova mètrica.

In [146]:
#Fem data mes petit agafant users random
# Modificat ja que al final hi havia un error, 
#he fet servir els meus mètodes de la pràctica
import random

data_ex3 = data.sample(n=500)

df_counts_ex3 = p2.build_counts_table(data_ex3)
sim_mtx_ex3 = p2.similarity_matrix_2(df_counts_ex3)

train_set, test_set = p2.get_sets(data_ex3, 0.1, 42)
assert len(test_set) + len(train_set) == len(data_ex3)


In [150]:
def evaluateRecommendations_mape(train, test, m, n, sim):
    """
    Retorna l'error generat pel model (MAPE)

    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAPE (en format decimal, no percentatge)
    """
    # el teu codi aquí
    errors = []

    # Iterem per cada usuari del test_set
    for user_id in test['user_id'].unique():

        # Obtenim les recomanacions per l'usuari
        recs = p2.getRecommendationsUser(train, user_id, sim, n, m)  # dataframe amb scores
        # Agafem les pelis que realment l'usuari ha vist al test
        test_user = test[test['user_id'] == user_id]

        # Només considerem les pelis que han estat recomanades
        merged = pd.merge(test_user, recs, on='movie_id', how='inner', suffixes=('_true', '_pred'))

        # Si no hi ha intersecció, saltem aquest usuari
        if merged.empty:
            continue

        # Calcul de l'error absolut
        fract_error = merged['rating'] - merged['predicted_score']
        fract_error = fract_error/merged['rating']
        abs_error = np.abs(fract_error)
        errors.extend(abs_error.tolist())

    # Mitjana de l'error absolut
    mape = np.mean(errors) if errors else np.nan
    return mape

In [151]:
# crea els conjunts de training i de test
# S'han creat adalt amb train_set, test_set = p2.get_sets(data_ex3, 0.1, 42)
train = train_set
test= test_set

In [152]:
# avaluació
evaluateRecommendations_mape(train, test, 50, 5, sim_mtx_ex3)

IndexError: index 2611 is out of bounds for axis 0 with size 458

In [154]:
df_counts = p2.build_counts_table(data)
sim_matrix = p2.similarity_matrix_2(df_counts)


# el teu codi aquí
errors = []

# Iterem per cada usuari del test_set
for user_id in test['user_id'].unique():

    # Obtenim les recomanacions per l'usuari
    recs = p2.getRecommendationsUser(data, user_id, sim_matrix, 5, 50)  # dataframe amb scores
    # Agafem les pelis que realment l'usuari ha vist al test
    test_user = test[test['user_id'] == user_id]

    # Només considerem les pelis que han estat recomanades
    merged = pd.merge(test_user, recs, on='movie_id', how='inner', suffixes=('_true', '_pred'))

    # Si no hi ha intersecció, saltem aquest usuari
    if merged.empty:
        continue

    # Calcul de l'error absolut
    fract_error = merged['rating'] - merged['predicted_score']
    fract_error = fract_error/merged['rating']
    abs_error = np.abs(fract_error)
    errors.extend(abs_error.tolist())

# Mitjana de l'error absolut
mape = np.mean(errors) if errors else np.nan
print(mape)

nan


### Pregunta

Com s'interpreten un MAE i un MAPE de 0.5 en el context del nostre recomanador?

### Resposta:





- Com s'interpreten per separat

    Amb la Mape vas sumant, percentualment, què tant semblants són els predicted ratings, és a dir, com valores aquesta recomanació, que tant aprop està de ser la valoració exacte


    Amb la MAE vas sumant què tant diferents són, però simplement fas la diferencia, és a dir, quants "punts" t'has allunyat de la resposta correcta

- Com s'interpreta el resultat

  
    Amb un 0'5 de MAE trobem que nomes ens allunyem en 0,5 punts de les dades reals (de mitjana


    Amb un 0'5 de MAPE trobem que les dades nomes s'assemblen a les reals en un 50% (de mitjana)